# Исследование надежности заемщиков

Необходимо определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от кредитного отдела банка — статистика о платёжеспособности клиентов.

## Откроем таблицу и изучим общую информацию о данных

**1. Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

**2. Выведем первые 20 строчек датафрейма data на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**4. Выведем количество пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**5. Заменим в столбце `total_income` пропущенные значения медианными значениями по каждому типу занятости из столбца `income_type`.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**6. В столбце `days_employed` присутствуют аномалии, которые скорее всего появились по ошибке. Заменим отризательные значения в столбце `days_employed` положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

**8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**12. Убедимся, что все пропуски заполнены.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**15. Выведем на экран количество строк-дубликатов в данных и удалим их при наличии.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**16. На основании диапазонов, указанных ниже, создим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных


**19. Есть ли зависимость между количеством детей и возвратом кредита в срок**

In [22]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


Вывод уникальных значений для столбца 'children':

In [23]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Получим количество заемщиков для каждого количества детей:

In [24]:
number_of_applicants_for_number_of_children = data.groupby('children')['children'].count()

Получим количество должников для каждого количества детей:

In [25]:
number_of_debtors_for_number_of_children = data.groupby('children')['debt'].sum()

Получим долю должников для каждого количества детей:

In [26]:
proportion_of_debtors_for_number_of_children = data.groupby('children')['debt'].mean()

Подготовим данные для отображения в табличном виде:

In [27]:
data_for_table = []

for index in range(len(data['children'].unique())):
    row_for_table = [index, number_of_applicants_for_number_of_children[index], 
                     number_of_debtors_for_number_of_children[index], 
                     proportion_of_debtors_for_number_of_children[index]]
    data_for_table.append(row_for_table)
    
column_names = ['Количество детей','Количество заемщиков', 'Количество должников','Доля должников']

Построим таблицу:

In [28]:
pd.DataFrame(data=data_for_table , columns=column_names)

,Количество детей,Количество заемщиков,Количество должников,Доля должников
0,0,14091,1063,0.075438
1,1,4808,444,0.092346
2,2,2052,194,0.094542
3,3,330,27,0.081818
4,4,41,4,0.097561
5,5,9,0,0.000000


**Вывод:**

По результатам полученных данных можно сделать следующий вывод:

1) Чем больше детей, тем реже люди берут кредиты;

2) Люди, у которых 5 детей, всегда возвращают кредиты в срок;

3) Заемщики, у которых нет детей или 3 ребенка, возвращают в срок чаще, чем заемщики, у которых 1 или 2 ребенка;

4) Чаще всего задерживаются с возвратом кредита люди, у которых 4 ребенка.

**20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Вывод уникальных значений для столбца 'family_status':

In [29]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Приведем их к нижнему регистру все значения столбца 'family_status':

In [30]:
data['family_status'] = data['family_status'].str.lower()

Получим количество заемщиков для каждого семейного положения:

In [31]:
number_of_applicants_for_types_of_family_regulations  = data.groupby('family_status')['family_status'].count()

Получим количество должников для каждого cемейного положения:

In [32]:
number_of_debtors_for_types_of_family_regulations = data.groupby('family_status')['debt'].sum()

Получим долю должников для каждого семейного положения:

In [33]:
proportion_of_debtors_for_types_of_family_regulations = data.groupby('family_status')['debt'].mean()

Подготовим данные для отображения в табличном виде:

In [34]:
data_for_table = []

family_status = data['family_status'].unique()
family_status.sort()


for index in range(len(data['family_status'].unique())):
    row_for_table = [family_status[index],
                     number_of_applicants_for_types_of_family_regulations[index], 
                     number_of_debtors_for_types_of_family_regulations[index], 
                     proportion_of_debtors_for_types_of_family_regulations[index]]
    data_for_table.append(row_for_table)
    
column_names = ['Семейное положение','Количество заемщиков', 'Количество должников','Доля должников']




Построим таблицу:

In [35]:
pd.DataFrame(data=data_for_table , columns=column_names)

,Семейное положение,Количество заемщиков,Количество должников,Доля должников
0,в разводе,1189,84,0.070648
1,вдовец / вдова,951,63,0.066246
2,гражданский брак,4134,385,0.093130
3,женат / замужем,12261,927,0.075606
4,не женат / не замужем,2796,273,0.097639


**Вывод:**

По результатам полученных данных можно сделать следующий вывод:
    
1) Реже всего берут кредиты люди с семейным положением 'вдовец / вдова'.
Также у данной категории людей самая маленькая доля должников.

Похожая ситуация у категории людей 'в разводе';

2) Чаще всего кредиты берут с семейный положением 'женат / замужем', однако доля должников у данной категории на третьем месте;

3) Чаще всего задерживают с возвратом заемщики с семейным положением 'не женат / не замужем' и 'гражданский брак'.

**21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Вывод уникальных значений для столбца 'total_income_category':

In [36]:
data['total_income_category'].unique()

array(['B', 'C', 'D', 'E', 'A'], dtype=object)

0–30000 — 'E'; 30001–50000 — 'D'; 50001–200000 — 'C'; 200001–1000000 — 'B'; 1000001 и выше — 'A'.

Получим количество заемщиков для каждого уровня дохода:

In [37]:
number_of_applicants_for_income_level = data.groupby('total_income_category')['total_income_category'].count()

Получим количество должников для каждого уровня дохода:

In [38]:
number_of_debtors_for_income_level = data.groupby('total_income_category')['debt'].sum()

Получим долю должников для каждого уровня дохода:

In [39]:
proportion_of_debtors_for_income_level = data.groupby('total_income_category')['debt'].mean()

Подготовим данные для отображения в табличном виде:

In [40]:
data_for_table = []

total_income_category = data['total_income_category'].unique()
total_income_category.sort()


for index in range(len(data['total_income_category'].unique())):
    row_for_table = [total_income_category[index],
                     number_of_applicants_for_income_level[index], 
                     number_of_debtors_for_income_level[index], 
                     proportion_of_debtors_for_income_level[index]]
    data_for_table.append(row_for_table)
    
column_names = ['Уровень дохода','Количество заемщиков', 'Количество должников','Доля должников']


Построим таблицу:

In [41]:
pd.DataFrame(data=data_for_table , columns=column_names)

,Уровень дохода,Количество заемщиков,Количество должников,Доля должников
0,A,25,2,0.080000
1,B,5014,354,0.070602
2,C,15921,1353,0.084982
3,D,349,21,0.060172
4,E,22,2,0.090909


**Вывод:**

По результатам полученных данных можно сделать следующий вывод:

1) Реже всего задерживают с возвратом люди с уровнем дохода категории D (30001–50000);

2) Следующие по задолжности являются заемщики с уровенем дохода категории B (200001–1000000);

3) Люди с уровнем дохода E (0–30000) и A (1000001 и выше) реже всего берут кредит, однако доли должников данной категории большие;

4) Заемщики с уровнем дохода C (50001–200000) чаще всего берут кредиты. Тем не менее доля должников данной категории большая.

**22. Как разные цели кредита влияют на его возврат в срок?**

Вывод уникальных значений для столбца 'purpose_category':

In [42]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

Получим количество заемщиков для каждой цели:

In [43]:
number_of_applicants_for_purpose = data.groupby('purpose_category')['purpose_category'].count()

Получим количество должников для каждой цели:

In [44]:
number_of_debtors_for_purpose = data.groupby('purpose_category')['debt'].sum()

Получим долю должников для каждой цели:

In [45]:
proportion_of_debtors_for_purpose = data.groupby('purpose_category')['debt'].mean()

Подготовим данные для отображения в табличном виде:

In [46]:
data_for_table = []

purpose_category = data['purpose_category'].unique()
purpose_category.sort()


for index in range(len(data['purpose_category'].unique())):
    row_for_table = [purpose_category[index],
                     number_of_applicants_for_purpose[index], 
                     number_of_debtors_for_purpose[index], 
                     proportion_of_debtors_for_purpose[index]]
    data_for_table.append(row_for_table)
    
column_names = ['Цель кредита','Количество заемщиков', 'Количество должников','Доля должников']

Построим таблицу:

In [47]:
pd.DataFrame(data=data_for_table , columns=column_names)

,Цель кредита,Количество заемщиков,Количество должников,Доля должников
0,операции с автомобилем,4279,400,0.093480
1,операции с недвижимостью,10751,780,0.072551
2,получение образования,3988,369,0.092528
3,проведение свадьбы,2313,183,0.079118


**Вывод:**

По результатам полученных данных можно сделать следующий вывод:

1) Чаще всего кредиты берут на операции с недвижимостью, тем не менее заемщики с данной целью чаще вызвращают деньги в срок;

2) Чуть реже, чем заемщики с целью осуществлении операции с недвижимостью, возвращают деньги в срок люди с целью проведения свадьбы;

3) Чаще всего задерживают с возвратом кредита в срок заемщики с целью получения образования и осуществления операции с автомобилем.

**23. Возможные причины появления пропусков в исходных данных.**

Ответ: По результатам задания 4 видно, что количество пропусков в столбце 'days_employed' (общий трудовой стаж в днях) совпадает с количеством пропусков в столбце 'total_income' (ежемесячный доход). Это может означать, что заемщиков отсутствует официальная работа и, соответственно, официальный доход.

**24. Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

Ответ: Пропуски количественных переменных следует заполнять медианным значением, так как данное значение будет являться самым типичным для данной выборки.

### Общий вывод

По результатам проведенного исследования можно сделать следующий вывод:

1) Люди, у которых 5 детей всегда возвращают кредиты в срок. 

Заемщики, у которых нет детей или 3 ребенка, возвращают в срок чаще, чем заемщики, у которых 1 или 2 ребенка.

Хуже всего отдают кредит люди, у которых 4 ребенка;


2) Вдовцы или люди в разводе отдают деньги в срок чаще, чем женатые.

Чаще всего задерживают с возвратом заемщики с семейным положением 'не женат / не замужем' и 'гражданский брак';


3) Реже всего задерживают с возвратом люди с уровнем дохода категории D (30001–50000) и B (200001–1000000).

Люди с уровнем дохода E (0–30000) и A (1000001 и выше) реже всего берут кредит, однако доли должников данной категории большие.

Заемщики с уровнем дохода C (50001–200000) чаще всего берут кредиты. Тем не менее доля должников данной категории большая;


4) Чаще всего кредиты берут на операции с недвижимостью, тем не менее заемщики с данной целью чаще вызвращают деньги в срок;

Чаще всего задерживают с возвратом кредита в срок заемщики с целью получения образования и осуществления операции с автомобилем.